In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

In [2]:
from synthetics import generate_single_task_unipolar, gaussian_bags_of_words, vocab1k

N = 1000
M = 30
K = 2
NUM_SPLITS = 3

Ds = [] # data
Xs = [] # features
Ls = [] # noisy labels
Ys = [] # true labels
for _ in range(NUM_SPLITS):
    L, Y, _ = generate_single_task_unipolar(
        N, M, k=K, acc=[0.6, 0.9], rec=[0.1, 0.2], 
        class_balance=[0.3, 0.7], lf_balance=None, seed=1)
    
    X, D = gaussian_bags_of_words(Y, vocab1k)
    
    Ls.append(L)
    Ys.append(Y)
    Ds.append(D)
    Xs.append(X)

In [3]:
search_space = {
    'n_epochs': 25,
    'l2': [0.0, 0.01, 0.1],
    'lr': {'range': [0.0001, 1], 'scale': 'log'},
}

In [9]:
from metal.label_model import LabelModel
from metal.tuner import ModelTuner

tuner = ModelTuner(LabelModel)
init_args = []
train_args = [Ls[0]]
model, best_config = tuner.search(init_args, train_args, Ls[1], Ys[1], 
                                  search_space, max_search=5, metric='f1')

[1] Testing {'l2': 0.01, 'lr': 0.0006945917330918604}
[Epoch 0] Loss: 0.180101
[Epoch 10] Loss: 0.179939
[Epoch 20] Loss: 0.179628
[Epoch 24] Loss: 0.179487
Finished Training
F1: 0.657
[2] Testing {'l2': 0.1, 'lr': 0.0018637133052883145}
[Epoch 0] Loss: 0.180101
[Epoch 10] Loss: 0.179671
[Epoch 20] Loss: 0.178875
[Epoch 24] Loss: 0.178527
Finished Training
F1: 0.656
[3] Testing {'l2': 0.0, 'lr': 0.028480062981020064}
[Epoch 0] Loss: 0.180101
[Epoch 10] Loss: 0.173526
[Epoch 20] Loss: 0.161283
[Epoch 24] Loss: 0.155877
Finished Training
F1: 0.672
[4] Testing {'l2': 0.01, 'lr': 0.00013961334707661579}
[Epoch 0] Loss: 0.180101
[Epoch 10] Loss: 0.180069
[Epoch 20] Loss: 0.180006
[Epoch 24] Loss: 0.179977
Finished Training
F1: 0.653
[5] Testing {'l2': 0.1, 'lr': 0.13801657369293308}
[Epoch 0] Loss: 0.180101
[Epoch 10] Loss: 0.166848
[Epoch 20] Loss: 0.168373
[Epoch 24] Loss: 0.167421
Finished Training
F1: 0.694
[SUMMARY]
Best model: [5]
Best config: {'n_epochs': 25, 'l2': 0.1, 'lr': 0.13801